# Pré processamento da coleção de dados

Aplicação de técnicas de pré processamento de dados para ser possível uma pré-análise dos dados enquanto ocorre a transformação para dados padronizados e normalizados.

# Parte 3 - Describe e Merge 

* foi utilizado a função describe() para ter a estatística dos dados de cada tabela tanto para os dados categóricos quanto para os dados discretos
* também unimos as tabelas de Interações (Detail_Interaction) e Incidentes (Detail_Incident)

In [1]:
from fun_dependencies import *

#load data
df_interacao = load_data('cleaning_dataset/pt2/Detail_Interaction_pt2.csv', sep=';')
df_incidente = load_data('cleaning_dataset/pt2/Detail_Incident_pt2.csv', sep=';')
df_atividades_incidente = load_data('cleaning_dataset/pt2/Detail_Incident_Activity_pt2.csv', sep=';')
df_mudancas = load_data('cleaning_dataset/pt2/Detail_Change_pt2.csv', sep=';')

#load raw data
df_raw_interacao = load_data('dataset/Detail_Interaction.csv', sep=';')
df_raw_incidente = load_data('dataset/Detail_Incident.csv', sep=';')
df_raw_atividade_incidente = load_data('dataset/Detail_Incident_Activity.csv', sep=';')
df_raw_mudancas = load_data('dataset/Detail_Change.csv', sep=';')

### Tabela Interacao

In [2]:
%%capture cap --no-stderr
print("---------------------------------------Table: Interacao---------------------------------------\n")
print(f"Statistics Numeric Columns: \n{df_interacao.describe(include=np.number)}\n")
print(f"Statistics Categorial Columns: \n{df_interacao.describe(exclude=np.number)}\n")

In [3]:
with open('output/output_pt3.txt', 'w') as f:
    f.write(cap.stdout)

### Tabela Incidente

In [4]:
%%capture cap --no-stderr
print("---------------------------------------Table: Incidente---------------------------------------\n")
print(f"Statistics Numeric Columns: \n{df_incidente.describe(include=np.number)}\n")
print(f"Statistics Categorial Columns: \n{df_incidente.describe(exclude=np.number)}\n")

In [5]:
with open('output/output_pt3.txt', 'a') as f:
    f.write(cap.stdout)

### Tabela Atividades Incidente

In [6]:
%%capture cap --no-stderr
print("---------------------------------------Table: Atividades Interacao---------------------------------------\n")
#print(f"Statistics Numeric Columns: \n{df_atividades_incidente.describe(include=np.number)}\n") # no numeric columns
print(f"Statistics Categorial Columns: \n{df_atividades_incidente.describe(exclude=np.number)}\n")

In [7]:
with open('output/output_pt3.txt', 'a') as f:
    f.write(cap.stdout)

### Tabela Mudanças

In [8]:
%%capture cap --no-stderr
print("---------------------------------------Table: Mudancas---------------------------------------\n")
#print(f"Statistics Numeric Columns: \n{df_mudancas.describe(include=np.number)}\n")  # no numeric columns
print(f"Statistics Categorial Columns: \n{df_mudancas.describe(exclude=np.number)}\n")

In [9]:
with open('output/output_pt3.txt', 'a') as f:
    f.write(cap.stdout)

# Merged Data

* Raw: Apply preprocessing after merge Detail Interaction + Detail Incident
* Processed: Apply preprocessing before merge Detail Interaction + Detail Incident

## Interaction + Incident

1. Merge
2. Missing
3. Formating
4. Describe

### Raw

In [10]:
df_raw_interacao_incidente_atividade = pd.concat([df_raw_incidente.set_index('incident_id'),df_raw_interacao.set_index('related_incident'),df_raw_atividade_incidente.set_index('incident_id')], join='outer')

df_raw_interacao_incidente_atividade = df_raw_interacao_incidente_atividade[df_raw_interacao_incidente_atividade.index.notnull()]

#df_raw_interacao_incidente.info()

In [11]:
%%capture cap --no-stderr
df_raw_interacao_incidente_atividade = df_raw_interacao_incidente_atividade.reset_index(col_level=0).rename(columns={'index': 'incident_id'})

df_raw_merge = pd.concat([df_raw_interacao_incidente_atividade.set_index('related_change'),df_raw_mudancas.set_index('change_id')], join='outer')

df_raw_merge = df_raw_merge.reset_index(col_level=0).rename(columns={'index': 'change_id'})

print('\nInfo after RAW merge:\n')
df_raw_merge.info()
print('\n')

In [12]:
with open('output/output_pt3.txt', 'a') as f:
    f.write(cap.stdout)

In [13]:
%%capture cap --no-stderr

## customize missing treatment

print("---------------------------------------RAW Table: Merged Interacao + Incidente + Atividade Incidente + Mudancas ---------------------------------------\n")
rows_before = df_raw_merge.shape[0]
before = pd.concat([df_raw_merge.isna().sum(), df_raw_merge.isna().sum()/len(df_raw_merge)*100], axis=1)
print(f'Before Missing Values\n{before}')

df_raw_merge.dropna(thresh=0.2*len(df_raw_merge), axis=1, inplace=True)
df_raw_merge.dropna(subset=['datestamp'], inplace=True)
df_raw_merge.dropna(thresh=1*len(df_raw_merge), axis=1, inplace=True)

rows_after = df_raw_merge.shape[0]
after = pd.concat([df_raw_merge.isna().sum(), df_raw_merge.isna().sum()/len(df_raw_merge)*100], axis=1)
print(f'\nAfter Missing Values\n{after}')
print('\nPercent missing value removed: {:.2%}\n'.format((rows_before-rows_after)/rows_before))

In [14]:
with open('output/output_pt3.txt', 'a') as f:
    f.write(cap.stdout)

In [15]:
%%capture cap --no-stderr
print(f'Attributes type\n{df_raw_merge.dtypes}\n')
df_raw_merge = data_formatting(df_raw_merge)
print(f'Attributes type\n{df_raw_merge.dtypes}\n')

In [16]:
with open('output/output_pt3.txt', 'a') as f:
    f.write(cap.stdout)

In [17]:
%%capture cap --no-stderr
#print(f"Statistics Numeric Columns: \n{df_raw_merge.describe(include=np.number)}\n")  # no numeric columns
print(f"Statistics Categorial Columns: \n{df_raw_merge.describe(exclude=np.number)}\n")

In [18]:
with open('output/output_pt3.txt', 'a') as f:
    f.write(cap.stdout)

In [19]:
# Sort by ascending incident_id,datestamp (case_id,timestamp), reset the indices
df_raw_merge = df_raw_merge.sort_values(by=['incident_id','datestamp','incidentactivity_number'])

### Processed

In [20]:
df_interacao_incidente_atividade = pd.concat([df_incidente.set_index('incident_id'),df_interacao.set_index('related_incident'),df_atividades_incidente.set_index('incident_id')], join='outer')

df_interacao_incidente_atividade = df_interacao_incidente_atividade[df_interacao_incidente_atividade.index.notnull()]

In [21]:
%%capture cap --no-stderr
df_merge = df_interacao_incidente_atividade.reset_index(col_level=0).rename(columns={'index': 'incident_id'})

#df_merge = pd.concat([df_interacao_incidente_atividade.set_index('related_change'),df_mudancas.set_index('change_id')], join='outer')

#df_merge = df_merge.reset_index(col_level=0).rename(columns={'index': 'change_id'})

print('\nInfo after PROCESSED merge:\n')
df_merge.info()
print('\n')

In [22]:
with open('output/output_pt3.txt', 'a') as f:
    f.write(cap.stdout)

In [23]:
%%capture cap --no-stderr
print("---------------------------------------PROCESSED Table: Merged Interacao + Incidente + Atividade Incidente + Mudancas ---------------------------------------\n")
df_merge = missing_treatment(df_merge, drop=False, fill=False, threshold=0.2)

In [24]:
with open('output/output_pt3.txt', 'a') as f:
    f.write(cap.stdout)

In [25]:
%%capture cap --no-stderr
print(f'Attributes type\n{df_merge.dtypes}\n')
df_merge = data_formatting(df_merge)
print(f'Attributes type\n{df_merge.dtypes}\n')

In [26]:
with open('output/output_pt3.txt', 'a') as f:
    f.write(cap.stdout)

In [27]:
%%capture cap --no-stderr
#print(f"Statistics Numeric Columns: \n{df_merge.describe(include=np.number)}\n")  # no numeric columns
print(f"Statistics Categorial Columns: \n{df_merge.describe(exclude=np.number)}\n")

In [28]:
with open('output/output_pt3.txt', 'a') as f:
    f.write(cap.stdout)

In [29]:
df_merge = df_merge.sort_values(by=['incident_id','datestamp','incidentactivity_number'])

In [30]:
df_raw_merge.to_csv('cleaning_dataset/pt3/RAW_MERGED_Detail_pt3.csv', sep=';', encoding='utf-8', index=False)
df_merge.to_csv('cleaning_dataset/pt3/PROCESSED_MERGED_Detail_pt3.csv', sep=';', encoding='utf-8', index=False)